In [3]:
import collections
from collections import Counter

#import helper
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline



import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split

#from tabulate import tabulate

import gc
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# loading
with open('tokenizerc.pickle', 'rb') as handle:
    tknizer_c = pickle.load(handle)
    # loading
with open('tokenizeruc.pickle', 'rb') as handle:
    tknizer_uc = pickle.load(handle)

In [5]:
vocab_size_uc=len(tknizer_uc.word_index.keys())
print(vocab_size_uc)
vocab_size_c=len(tknizer_c.word_index.keys())
print(vocab_size_c)

3008
7500


In [6]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

      super().__init__()
      self.inp_vocab_size= inp_vocab_size
      self.embedding_size = embedding_size
      self.input_length = input_length
      self.lstm_size = lstm_size
      self.lstm_output = 0
      self.lstm_state_h = 0
      self.lstm_state_c = 0
      self.embedding = Embedding(input_dim=self.inp_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                            mask_zero=True, name="embedding_layer_encoder")
      self.lstm = LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence,states):
      input_embedd= self.embedding(input_sequence)
      self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
      return self.lstm_output, self.lstm_state_h,self.lstm_state_c


    
    def initialize_states(self,batch_size):
      #check
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      initial_hidden_state =tf.zeros(shape=[batch_size,self.lstm_size],dtype=tf.int32)
      initial_cell_state =tf.zeros(shape=[batch_size,self.lstm_size],dtype=tf.int32)
      return initial_hidden_state,initial_cell_state

In [7]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):
    super().__init__()
    self.scoring_function = scoring_function
    self.att_units = att_units

    # Please go through the reference notebook and research paper to complete the scoring functions

    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      # encoder output
      self.Dot = tf.keras.layers.Dot(axes=(1, 2))
    if scoring_function == 'general':
      self.W1= tf.keras.layers.Dense(att_units)
      self.Dot = tf.keras.layers.Dot(axes=(1, 2))
    elif scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
        self.W2 = tf.keras.layers.Dense(att_units)
        self.W3 = tf.keras.layers.Dense(att_units)
        self.V = tf.keras.layers.Dense(1)
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        decoder_hidden_state = tf.expand_dims(decoder_hidden_state,1)
        dhs = tf.transpose(decoder_hidden_state,(0,2,1))
        #print(dhs.shape)
        #print(encoder_output.shape)
        x = self.Dot([dhs,encoder_output])
        a = tf.nn.softmax(tf.transpose(x,(0,2,1)),axis=1)
        c = a*encoder_output
        c = tf.reduce_sum(c,axis = 1)
        #print(c.shape,a.shape)
        return c,a




  


    elif self.scoring_function == 'general':
        # Implement General score function here
        decoder_hidden_state = tf.expand_dims(decoder_hidden_state,1)
        wa = self.W1(encoder_output)
        dhs = tf.transpose(decoder_hidden_state,(0,2,1))
        x = self.Dot([dhs,wa])
        
        a = tf.nn.softmax(tf.transpose(x,(0,2,1)),axis=1)
        c = a*encoder_output
        c = tf.reduce_sum(c,axis = 1)
        #print(c.shape,a.shape)
        return c,a

    elif self.scoring_function == 'concat':
        # Implement General score function here
        decoder_hidden_state = tf.expand_dims(decoder_hidden_state,1)
        e = self.V(tf.nn.tanh(self.W2(decoder_hidden_state)+self.W3(encoder_output)))
        a = tf.nn.softmax(e,axis = 1)
        c = a*encoder_output
        c = tf.reduce_sum(c,axis = 1)
        m = self.W2
        #print(c.shape,a.shape)
        #print(m.shape)
        return c,a

In [8]:
class One_Step_Decoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
    super().__init__()
    self.tar_vocab_size = tar_vocab_size 
    self.embedding_dim = embedding_dim
    self.input_length =  input_length
    self.dec_units = dec_units
    self.score_fun = score_fun
    self.att_units = att_units
# Initialize decoder embedding layer, LSTM and any other objects needed
    self.lstm_output = 0
    self.lstm_state_h = 0
    self.lstm_state_c = 0
    self.embedding = Embedding(input_dim=self.tar_vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                            mask_zero=True, name="embedding_layer_encoder")
    self.lstm = LSTM(att_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
    self.dense   = Dense(self.tar_vocab_size)
    self.attention = Attention(scoring_function = self.score_fun,att_units = self.att_units)




  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    
    input_embedding= self.embedding(input_to_decoder)
    context,similarities = self.attention(state_h,encoder_output)
    #print(input_embedding.shape)
    #print(context.shape)
    concat = tf.keras.layers.Concatenate(axis = 2)([input_embedding,tf.expand_dims(context,1)])
    decoder_output,*states = self.lstm(concat)
    
    #print(states[0].shape)
    decoder_hidden,decoder_cell = states[0],states[1]

    output =self.dense(decoder_output[:,:,:self.dec_units])
    #print(decoder_hidden.shape)
    return tf.squeeze(output,axis = 1),decoder_hidden,decoder_cell,similarities,context
#context_vector = 32116
#i2d_output = 32112

In [9]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
                
      super().__init__()
      #Intialize necessary variables and create an object from the class onestepdecoder
      self.out_vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim

      self.input_length = input_length
      self.dec_units = dec_units
      self.score_fun = score_fun
      self.att_units = att_units
      self.onestepdecoder=One_Step_Decoder(self.out_vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)  

        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        all_outputs = tf.TensorArray(tf.float32, size=40, name = 'output_arrays',dynamic_size = True,clear_after_read = False)
        for i in range(40):
          output,decoder_hidden_state ,decoder_cell_state,attention_weights,context_vector = self.onestepdecoder(input_to_decoder[:,i:i+1],encoder_output,decoder_hidden_state,decoder_cell_state)#sequence is correct
          all_outputs = all_outputs.write(i,output)

       # all_outputs.mark_used()


        all_outputs = tf.transpose(all_outputs.stack(),[1,0,2])
        return all_outputs

In [10]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,encoder_inputs_length,decoder_inputs_length,output_vocab_size,batch_size):
    #Intialize objects from encoder decoder
    super().__init__()
    self.encoder = Encoder(inp_vocab_size = vocab_size_c+2,embedding_size = 100,lstm_size = 512,input_length = encoder_inputs_length)
    self.decoder = Decoder(out_vocab_size = vocab_size_uc+2,embedding_dim = 100,input_length = decoder_inputs_length ,dec_units = 512,score_fun = 'concat',att_units = 512)
    self.dense   = Dense(output_vocab_size, activation='softmax')
    self.batch_size = batch_size
    self.embedding = Embedding(input_dim=vocab_size_c+1, output_dim=100, input_length=40,
                           mask_zero=True, name="embedding_layer_encoder")

  
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    input,output = data[0],data[1]
    initial_state = self.encoder.initialize_states(self.batch_size)
    input_embedd= input
    encoder_output,encoder_h,encoder_c = self.encoder(input_embedd,initial_state)
    decoder_output = self.decoder(output,encoder_output,encoder_h,encoder_c)
    return decoder_output
   

In [11]:
#https://www.tensorflow.org/tutorials/text/image_captioning#model
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


def loss_function(real, pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple sparse categorical crossentropy?
    Yes, you can use simple sparse categorical crossentropy as loss like we did in task-1. But in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [12]:
model  = encoder_decoder(encoder_inputs_length=40,decoder_inputs_length=40,output_vocab_size=vocab_size_uc,batch_size =1)


In [13]:
model.built = True

In [23]:
model.load_weights('my_model2')

In [24]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         Save the attention weights
         And get the word using the tokenizer(word index) and then store it in a string.
  E. Call plot_attention(#params)
  F. Return the predicted sentence
  '''
  attention_plot = np.zeros((40,40))
  input_sequences = tknizer_c.texts_to_sequences([input_sentence])
  encoder_sequence = pad_sequences(input_sequences, maxlen = 40, dtype='int32', padding='post')
  encoder_output,state_h,state_c= model.layers[0](encoder_sequence,model.layers[0].initialize_states(1))
  dec_input = tknizer_uc.word_index['<start>']
  dec_input = (np.array(dec_input)).reshape(1,1)
  states = [state_h,state_c]
  j = 0
  final = []
  while j!=40:
    predicted_out,state_h,state_c,attention_weights,context_vector=model.layers[1].onestepdecoder(dec_input,encoder_output,state_h,state_c)
    #att_w.append(state_c)
    
    attention_weights = tf.reshape(attention_weights,(-1,))
    attention_plot[j] = attention_weights.numpy()
    #______________________________________

    output = predicted_out
    states = [state_h,state_c] #automatically get updated as above
    output= tf.argmax(output[0]).numpy()
    #print(output)
    b = get_key(output)
    if b == '<end>':
      break
    else:
      final.append(get_key(output))
      dec_input =tf.expand_dims([output], 0)
      j+=1
 # plot_attention(att_w,input_sentence,final)
  
  return ' '.join(final),input_sentence,attention_plot

In [25]:
def get_key(val):
    for key, value in tknizer_uc.word_index.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [26]:
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

def preprocess(text):
    # convert all the text into lower letters
    # use this function to remove the contractions: https://gist.github.com/anandborad/d410a49a493b56dace4f814ab5325bbd
    # remove all the spacial characters: except space ' '
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    return text

def preprocess_c(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'
    # we have found these characters after observing the data points, feel free to explore more and see if you can do find more
    # you are free to do more proprocessing
    # note that the model will learn better with better preprocessed data 
    
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text


In [27]:
x = pd.read_csv("File1")

In [28]:
def Final(x):
  z = []
  dfx  = x
  dfx = dfx.drop("Unnamed: 0",axis = 1)
  dfx = dfx.drop("Unnamed: 0.1",axis = 1)
  c  = dfx.columns[0]
  dfx["c_l"] =  dfx[c].str.split().apply(len)
  dfx = dfx[dfx['c_l'] < 40]
  dfx = dfx.drop(['c_l'],axis = 1)
  dfx[c] = dfx[c].apply(preprocess_c)
  for j in range(len(dfx)):
        f,i,at  =  predict(dfx.iloc[j][0])
        print('Orignal Corrupted : ',dfx.iloc[j][0])
        print('Orignal  Uncorrupted : ',dfx.iloc[j][1])
        print('Predicted Corrected: ',f)
  


In [29]:
Final(x)

Orignal Corrupted :  eh  ü all go to ben  s place on sat 
Orignal  Uncorrupted :  <start> huh you all going to ben is place on saturday
Predicted Corrected:  huh you all going to be a good night and then is place or not on monday and then is place or not on monday and then is place or not on monday and then is place or not on
Orignal Corrupted :  hey  tonight move online at 10 we make up stuff for friday
Orignal  Uncorrupted :  <start> hey tonight go online at 10 we settle stuff for friday
Predicted Corrected:  hey tonight go online go and read the number and rebecca reserve seats
Orignal Corrupted :  sorry for belated reply wuz damning ü some recruit juz like a shot he frm acj lei my english cant  t be besides ornate iv him to understan blood furuncle  go bathe 1st
Orignal  Uncorrupted :  <start> sorry for late reply what is damning you some recruit just now he is from acj my english cant not be too flowery for him to understand blood boil go bathing first
Predicted Corrected:  sorry 

Orignal Corrupted :  haha ok   how late i go buy just the ticket foremost
Orignal  Uncorrupted :  <start> haha okay how late i go to buy tickets first
Predicted Corrected:  haha okay how late i go to find one is late the rest early right i go to find one is late the rest early right i go to find one is late the rest early right i go to
Orignal Corrupted :  i recollect si 29 one haven exercise  must ask ü how to employ excel succeeding time ü glad new year to ü overly atomic number 53 ask ard n confirm wif ü the date
Orignal  Uncorrupted :  <start> i think it is 29 i have not done i must ask you how to use excel next time you happy new year to you too i ask around and confirm with you the date
Predicted Corrected:  i think it is 29 i have not done i must ask you how to use excel next time you happy new year to you too i ask around and confirm with you the date
Orignal Corrupted :  aloha state boy single m a old female person lady
Orignal  Uncorrupted :  <start> hi boy i am an old female

Orignal Corrupted :  oh i managed to get the off cos i did duty on sat some time ago soyou r going shopping wif yr frens
Orignal  Uncorrupted :  <start> oh i managed to get the off because i did duty on saturday some time ago so are you going shopping with your friends
Predicted Corrected:  oh i managed to get the off because i am going to take care of this month yes go to take care of this month yes go to take care of this month yes go to take care of this
Orignal Corrupted :  ohtt guy who is much taller than me ritehmmthis current one is slightly taller than me onlyhmmyest got someone msged me ñ said he was li en ñ tt tt was
Orignal  Uncorrupted :  <start> oh that guy who is much taller than me right hmm this current one is slightly taller than me only hmm yesterday got someone messaged me and said he was li en and that that was
Predicted Corrected:  of course is tonight
Orignal Corrupted :  call 93864500 ñ ask if they got any telemarketin vacancy tis be the hdb hub ace it  s job bri

Orignal Corrupted :  haha  sooner state  certain  hehe  must buy affair for pine tree state hor keke  if fuck off go stunned eat farewell meal with u keke
Orignal  Uncorrupted :  <start> haha okay sure hehe must buy thing for me hehe if go out to eat farewell meal with you hehe
Predicted Corrected:  haha still need 58 tomorrow you must go out for shopping
Orignal Corrupted :  or ü can break down toa payoh central  so ü send away shop shop 
Orignal  Uncorrupted :  <start> or you can go to toa payoh central then you can shop and shop
Predicted Corrected:  or you can go and come and wear it can go and come and wear it can go and come and wear it can go and come and wear it can go and come and wear it can go and
Orignal Corrupted :  we regret to inform atomic number 92 that the nhs hour angle throw a misunderstanding u were never actually max born please report 2 yor local hospital 2b terminated we r sorry 4 the inconvenience
Orignal  Uncorrupted :  <start> we regret to inform you that the

Orignal Corrupted :  lancome lcm foam cleanse gel combination to dry hide 
Orignal  Uncorrupted :  <start> lancome  lcm foaming cleansing gel combination to dry skin
Predicted Corrected:  lancome lcm foaming cleansing gel combination to dry skin
Orignal Corrupted :  hmm  nvm lor  get frm u another dae  = 
Orignal  Uncorrupted :  <start> hmm nevermind then get from you another day
Predicted Corrected:  hmm nevermind then get from you another day
Orignal Corrupted :  daddy at home what time you leave be back to get maine to airport
Orignal  Uncorrupted :  <start> daddy is at home what time will you be back to fetch me to airport
Predicted Corrected:  daddy is at the way what you be back to go maldives now i go to book for the way what you be back to go maldives now i go to book for the way what you be back to
Orignal Corrupted :  aft year process la  dunno where  townsfolk
Orignal  Uncorrupted :  <start> after your work do not know where town
Predicted Corrected:  after your dinner i thi

Orignal Corrupted :  nvr log off la juz move e screen only
Orignal  Uncorrupted :  <start> never log off just move the screen only
Predicted Corrected:  never log just bring my hair is friends
Orignal Corrupted :  hi mate its rv did u hav a nice hol just a message 3 say hello coz haven  t send u 1 in ages start tug thus stay remove roadstead rvx
Orignal  Uncorrupted :  <start> hi mate it is rv did you have a nice holiday just a message to say hello because i have not sent you one in ages  i started driving so stay off roads
Predicted Corrected:  hi mate it is rv did you have a nice holiday just a message to say hello because i have not sent you free to a pub just got it was a really nice holiday just a message to say
Orignal Corrupted :  game for a drink subsequently
Orignal  Uncorrupted :  <start> game for a drink later
Predicted Corrected:  game for you back for a drink later for a drink later for a drink later for a drink later for a drink later for a drink later for a drink later f

Orignal Corrupted :  ask to a greater extent abt me
Orignal  Uncorrupted :  <start> ask more about me
Predicted Corrected:  he is going through tutorial hehe to have a lot of the last lecture
Orignal Corrupted :  ah  realli eh  okeh  i see wat single crapper do k let uranium know afterward
Orignal  Uncorrupted :  <start> really ok i see what i can do ok let you know later
Predicted Corrected:  really ok i see what i can do ok let you know later
Orignal Corrupted :  nope   you external
Orignal  Uncorrupted :  <start> no you outside
Predicted Corrected:  no
Orignal Corrupted :  haha  sorrie abit blurr liao dynasty  havin too much lesson  so hows life for atomic number 92
Orignal  Uncorrupted :  <start> haha sorry i am a bit blur i am having too much lesson so how is life for you
Predicted Corrected:  haha sorry i am having too you are you just now you are you just now you are you just now you are you just now you are you just now you are you just now you are you just
Orignal Corrupted : 

Orignal Corrupted :  haha  cause supervisor go oversea lo so no one look after pine tree state lo  hehe but one however get find matter do lo
Orignal  Uncorrupted :  <start> haha because supervisor went overseas then no one looks after me hehe but i still find thing to do
Predicted Corrected:  haha because supervisor went overseas then no one for you tomorrow thanks for me hehe but i still find thing to do
Orignal Corrupted :  opps  sorry  i keep tink tmr be wed  sorry  haha
Orignal  Uncorrupted :  <start> oops sorry i kept thinking tomorrow is wednesday sorry haha
Predicted Corrected:  oops sorry i think she said haha by the way is 180 to 200 the late i think she said haha by the way is 180 to 200 the late i think she said haha by the way is 180
Orignal Corrupted :  yun ah  uranium go 4 stats tmr 1 nid to copy notes frm u  = 
Orignal  Uncorrupted :  <start> yun you are going for statistics tomorrow i need to copy notes from you
Predicted Corrected:  yun you want to go for statistics t

KeyboardInterrupt: 